In [8]:
import sys
sys.path.append("C:/Users/ahmednadiyah/Downloads/final_codes/ts-mule_main")
sys.path.append("C:/Users/ahmednadiyah/Downloads/final_codes/ts-mule/Window_Shap")
import dill

import numpy as np
import pandas as pd
import shap
import tensorflow as tf
import timeit
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from tsmule.xai.lime import LimeTS
from tsmule.sampling.segment import WindowSegmentation, MatrixProfileSegmentation, SAXSegmentation
from tsmule.sampling.perturb import Perturbation
from tsmule.xai.evaluation import PerturbationAnalysis
import windowshap 

In [3]:
dnn_model = tf.keras.models.load_model('C:/Users/ahmednadiyah/Downloads/final_codes/beijing_air_multi_site_models/beijing_air_multi_site_dnn_model.h5', compile=False)

with open('C:/Users/ahmednadiyah/Downloads/final_codes/beijing_air_multi_site_models/beijing_air_multi_site_test_data.dill', 'rb') as f:
    dataset_test = dill.load(f)

In [4]:
sample=dataset_test[0] 
labels=dataset_test[1]

In [4]:
print(sample.shape)
labels.shape

(38186, 24, 11)


(38186,)

In [5]:
def predict_fn(x):
    if len(x.shape) == 2:
        prediction = cnn_model.predict(x[np.newaxis, :, :]).ravel()
    else:
        prediction = cnn_model.predict(x).ravel()
    return prediction

#### Explanations with SHAP

In [9]:
tic=timeit.default_timer()
shap_exp=shap.DeepExplainer(dnn_model, sample[:3000]) #expected shape (None, 24, 7)

shap_values=shap_exp.shap_values(sample)
    
print('Total time: ' + str(timeit.default_timer()-tic))


Total time: 3938.6838848


In [10]:
shap_values=np.asarray(shap_values).squeeze()
shap_values.shape

(38186, 24, 11)

In [11]:
with open("C:/Users/ahmednadiyah/Downloads/multi_site_dnn_shap_values.dill", 'wb') as f:
    dill.dump(shap_values, f)

### Explanation with LIME

In [6]:
lasso_classifier = linear_model.Lasso(alpha=0.01)  #faster the model, faster LIME works
per=Perturbation() #perturbation object for LimeTS object, sampler

#### Explanation with LIME and uniform window segmentation

In [ ]:
#LimeTS object for uniform segmentation
tic = timeit.default_timer()
#segments object, WindowSegmentation object has stationery and exponential segmentations techniques 
uniform_seg=WindowSegmentation(partitions=4, win_length=24) 
uniform_lime=LimeTS(kernel=lasso_classifier, segmenter=uniform_seg, sampler=per, n_samples=24)
lime_values_uni=[uniform_lime.explain(sample[i], predict_fn, segmentation_method='uniform')
                 for i in range(len(sample))]
print('Total time: ' + str(timeit.default_timer()-tic))

1/1 [==============================] - 0s 16ms/step


In [ ]:
with open("C:/Users/ahmednadiyah/Downloads/Beijing_air_quality_dnn_lime_values_uni.dill", 'rb') as f:
    dill.dump(lime_values_uni, f)

#### Explanation with exponential window segmentation

In [ ]:
#LimeTS object for exponential window segmentation
tic = timeit.default_timer()
#segment object, WindowSegmentation has stationery and exponentials segmentation techniques
exp_seg=WindowSegmentation(partitions=4, win_length=24)
exp_lime=LimeTS(kernel=lasso_classifier, segmenter=exp_seg, sampler=per, n_samples=24)
#explainer for LimeTS
lime_values_exp=[exp_lime.explain(sample[i], predict_fn, segmentation_method='exponential')
                 for i in range(len(sample))]

print('Total time: ' + str(timeit.default_timer()-tic))

In [ ]:
with open("C:/Users/ahmednadiyah/Downloads/Beijing_air_quality_dnn_lime_values_exp.dill", 'rb') as f:
    dill.dump(lime_values_exp, f)

#### Explanation with MatrixProfile Window segmentation- slopes-sorted

In [ ]:
#LimeTS object for matrix profile segmentation= slopes-sorted
tic = timeit.default_timer()
#segment object, MatrixProfileSegmentation object has slopes, bins-min, bins-max segmentation techniques 
seg_slopes=MatrixProfileSegmentation(partitions=4, win_length=20)

lime_slopes=LimeTS(kernel=lasso_classifier, segmenter=seg_slopes, sampler=per)
lime_values_slopes=[lime_slopes.explain(sample[i], predict_fn,
                                                  segmentation_method='slopes-sorted') for i in
                    range(len(sample))]

print('Total time: ' + str(timeit.default_timer()-tic))

In [ ]:
with open("C:/Users/ahmednadiyah/Downloads/Beijing_air_quality_dnn_lime_values_slopes.dill", 'rb') as f:
    dill.dump(lime_values_slopes, f)

#### Explanation with SAX Transformation

In [ ]:
#LimeTS object for SAX segmentation
tic = timeit.default_timer()
#create segment object for SAX Transformation 
seg_sax=SAXSegmentation(partitions=4, win_length=10)

lime_sax=LimeTS(kernel=lasso_classifier, segmenter=seg_sax, sampler=per, n_samples=24)
lime_values_sax=[lime_sax.explain(sample[i], predict_fn) for i in range(len(sample))] 

print('Total time: ' + str(timeit.default_timer()-tic))

In [ ]:
with open("C:/Users/ahmednadiyah/Downloads/Beijing_air_quality_dnn_lime_values_sax.dill", 'rb') as f:
    dill.dump(lime_values_sax, f)